### Pre-Seeded Chat Bot Using OpenAI API
- In this notebook I am playing around with how you could go about pre-seeding a conversation to provide a better response to the user.

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

Forcing Dark mode in Gradio

In [2]:
force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

Load in Keys

In [3]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-


Initialize the Model

In [4]:
openai = OpenAI()
MODEL = 'gpt-4o-mini'

Message GPT Function

In [5]:
# def message_gpt(prompt):
#     messages = [
#         {"role": "system", "content": system_message},
#         {"role": "user", "content": prompt}
#       ]
#     completion = openai.chat.completions.create(
#         model='gpt-4o-mini',
#         messages=messages,
#     )
#     return completion.choices[0].message.content

Modified Message_GPT function to allow prior conversation seeding.

In [6]:
def message_gpt(prompt, seed_conversation=None):
    messages = [{"role": "system", "content": system_message}]
    
    if seed_conversation:
        messages.extend(seed_conversation)  # Add pre-seeded conversation
    
    messages.append({"role": "user", "content": prompt})
    
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )
    return completion.choices[0].message.content


In [7]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

Chat Function with Conversation Seeding

In [8]:
def chat(message, history):
    # Define a pre-seeded conversation
    seed_conversation = [
        {"role": "user", "content": "Hello, I am looking to buy some clothes?"},
        {"role": "assistant", "content": "I am a helpful AI Clothing specialist at this clothing store."},
        {"role": "user", "content": "Can you tell me some of the prices of the clothing that are on sale?"},
        {"role": "assistant", "content": "The majority of our hats range from $15 to $20, shoes are $50 to $60, and we don't have prices for pants because we do not sell pants."}
    ]

    # Determine the appropriate system message
    relevant_system_message = system_message
    if "belt" in message:
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    # Combine system message, pre-seeded conversation, chat history, and new message
    messages = [{"role": "system", "content": relevant_system_message}] + seed_conversation + history + [{"role": "user", "content": message}]

    # Create a streaming response from OpenAI
    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response


#### Chat Function with Multiple different Conversation Seeds
- After playing around with this I can tell that the system messages would need to be chosen carefully if you want to have a versatile chatbot for multiple different tasks

In [11]:
def chat(message, history):
    # Define multiple seed conversations
    seed_conversations = {
        "general": [
            {"role": "user", "content": "Hello, who are you?"},
            {"role": "assistant", "content": "I am a helpful AI assistant here to assist you."}
        ],
        "shopping": [
            {"role": "user", "content": "What products do you sell?"},
            {"role": "assistant", "content": "We sell a variety of items, but no belts."}
        ],
        "tech_support": [
            {"role": "user", "content": "Can you help me troubleshoot my computer?"},
            {"role": "assistant", "content": "Of course! What seems to be the issue?"}
        ]
    }

    # Determine the relevant seed conversation based on message content
    if "buy" in message or "product" in message:
        selected_seed = seed_conversations["shopping"]
    elif "computer" in message or "troubleshoot" in message:
        selected_seed = seed_conversations["tech_support"]
    else:
        selected_seed = seed_conversations["general"]

    # Determine the appropriate system message
    relevant_system_message = system_message
    if "belt" in message:
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    # Combine system message, selected seed conversation, chat history, and new user message
    messages = [{"role": "system", "content": relevant_system_message}] + selected_seed + history + [{"role": "user", "content": message}]

    # Create a streaming response from OpenAI
    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response


Check Gradio Version

In [9]:
# !pip show gradio
# 5.8.0 to 5.12

In [12]:
gr.ChatInterface(fn=chat, type="messages", js=force_dark_mode).launch()

* Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


Note: The LLM is "very gently encouraging" us to buy hats that are on sale, which is quite funny.